In [ ]:
import pandas as pd
import ast
import gdown
import numpy as np

## 1. TRANSFORMACIONES

In [ ]:
'''1.0 Crear acceso a dataframe a partir de las bases de datos:movies_dataset.csv y credits.csv '''
url_movies = 'https://drive.google.com/u/0/uc?id=1Rp7SNuoRnmdoQMa5LWXuK4i7W1ILblYb&export=download'
url_credits = 'https://drive.google.com/uc?id=1lMGJUWVVVRPO00ZWqzEJZIxFhSqtfmAB&export=download'

csv_movies = 'movies_dataset_original01.csv'
csv_credits = 'credits_original01.csv'

gdown.download(url_movies, csv_movies, quiet=False, format='csv')
gdown.download(url_credits, csv_credits, quiet=False, format='csv')

df = pd.read_csv(csv_movies, low_memory=False)
df_credits = pd.read_csv(csv_credits, low_memory=False)

In [ ]:
# -----------
''' 1.2 y 1.3. Manejo de valores nulos:
    Reemplaza los valores nulos por el valor 0'''
df['revenue'] = df['revenue'].fillna(0) 
df = df.dropna(subset=['release_date'])  # Elimina las filas del dataframe para las que no hay fecha de lanzamiento 
''' El campo budget tiene registros que no son números, p.e. "jsfhaksh.jpg", por ello se usa el parámetro errors = 'coerce' 
    el cual escribe NaN en los renglones que no contienen numeros y que por lo tanto no se pueden convertir a float, 
    luego reemplaza los NaN con 0'''
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].fillna(0) 

# -----------
''' 1.4. Manejo de FECHAS
    Campos con fechas: release_date'''
df['release_date'] = pd.to_datetime(df['release_date'], errors = 'coerce')
df['release_year'] = df['release_date'].dt.strftime('%Y')

# -----------
'''1.5. Calcular retorno de inversión
    Crea un nuevo dataframe para aplicar la operación requerida, recorriendo cada renglón y aplicando un criterio para 
    el caso de que budget o revenue sean 0 o Nan.
    Nuevo DF con las variables revenue y budget: '''
rb = pd.DataFrame({'REVENUE': df['revenue'], 'BUDGET': df['budget']})
''' Calcula el retorno. La palabra "return" esta reservada, por eso la variable se llamará "return_rate", aunque el titulo de 
    la columna si sea "return"'''
return_rate = rb.apply(lambda row: row['REVENUE'] / row['BUDGET']
                        if (row['BUDGET'] != 0) & (row['REVENUE'] != 0) else 0, axis=1)
''' Agrega la columna "return" al dataframe y lo rellena con los datos de return_rate'''
df['return'] = return_rate

# -----------
''' 1.6. Elimina columnas que no se van a necesitar
    Lista con los nombres de las columnas que se van a eliminar del dataset movies'''
col_to_elimin = ['video','imdb_id', 'adult','original_title','poster_path','homepage']
'''     Elimina del dataframe las columnas especificadas'''
df = df.drop(col_to_elimin, axis='columns')

'''Del dataset credits: elimina la información innecesaria dentro de los diccionarios'''
'''Columna cast: elimina keys y valores que no se usan en las funciones de la API'''
def eliminar_claves_valor(diccionarios):
    for diccionario in diccionarios:
        for clave in ['cast_id', 'character', 'credit_id', 'gender', 'order', 'profile_path']:
            diccionario.pop(clave, None)
    return diccionarios
'''Aplicar la función a la columna 'cast'''
df_credits['cast'] = df_credits['cast'].apply(eliminar_claves_valor)
'''Columna crew: elimina keys y valores que no se usan en las funciones de la API'''
def eliminar_claves_valor(diccionarios):
    for diccionario in diccionarios:
        for clave in ['credit_id', 'department', 'gender', 'profile_path']:
            diccionario.pop(clave, None)
    return diccionarios
'''Aplicar la función a la columna 'crew'''
df_credits['crew'] = df_credits['crew'].apply(eliminar_claves_valor)
'''Borra los diccionarios que no contienen nombres de directores en la columna crew'''
'''Función para seleccionar los diccionarios que tienen un valor específico en la key 'job'''
def eliminar_diccionario(diccionarios, valor):
    return [diccionario for diccionario in diccionarios if diccionario.get('job') == valor]
'''Valor a buscar en la clave 'key2'''
valor_a_conservar = 'Director'
''' Aplicar la función a la columna 'crew'''
df_credits['crew'] = df_credits['crew'].apply(lambda x: eliminar_diccionario(x, valor_a_conservar))

# -----------
'''1.7. EXTRA: Asignar tipos de datos correspondientes:'''
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')
df['id'] = pd.to_numeric(df['id'], errors='coerce')

# -----------
''' 1.8. EXTRA: Hacer join o merge entre los dos datasets para hacer las consultas en la API.
    Se hace in inner join, ya que solo se van a ocupar los registros que tengan coincidencias en ambos datasets'''
completo = pd.merge(df, df_credits, on='id')



In [ ]:
completo.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return,cast,crew
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,12.451801,"[{'id': 31, 'name': 'Tom Hanks'}, {'id': 12898...","[{'job': 'Director', 'name': 'John Lasseter'}]"
1,{},65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995.0,4.043035,"[{'id': 2157, 'name': 'Robin Williams'}, {'id'...","[{'job': 'Director', 'name': 'Joe Johnston'}]"
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995.0,0.000000,"[{'id': 6837, 'name': 'Walter Matthau'}, {'id'...","[{'job': 'Director', 'name': 'Howard Deutch'}]"
3,{},16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995.0,5.090760,"[{'id': 8851, 'name': 'Whitney Houston'}, {'id...","[{'job': 'Director', 'name': 'Forest Whitaker'}]"
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862.0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995.0,0.000000,"[{'id': 67773, 'name': 'Steve Martin'}, {'id':...","[{'job': 'Director', 'name': 'Charles Shyer'}]"


### EXPORTAR A CSV LOS DATASET NECESARIOS PARA LAS SIGUIENTES FASES DEL PROYECTO (EDA Y ML)

In [ ]:
# -----------
'''Dataset MOVIES FILTRADO. Crea una lista con los nombres de las columnas que se van a eliminar y luego las elimina con drop. Exporta el
dataframe como un archivo csv que se guarda en el workspace'''
elim_col_df = ['belongs_to_collection','budget','genres','id','original_language','overview','production_companies',
               'production_countries','revenue','runtime','spoken_languages','status','tagline','return']
movies = df.drop(elim_col_df, axis=1)
movies.to_csv('movies_dataset_filtrado_RMCE1.csv', index=False)

'''Dataset CAST. Crea una lista con los nombres de las columnas que se van a eliminar y luego las elimina con drop. Exporta el
dataframe como un archivo csv que se guarda en el workspace'''
elim_col_actor = ['belongs_to_collection','genres','id','original_language','overview','popularity','production_companies',
                     'production_countries', 'release_date','runtime','spoken_languages','status','tagline','vote_average',
                     'vote_count','release_year','crew']
movies_cast_actor = completo.drop(elim_col_actor,axis=1)
movies_cast_actor.to_csv('movies_cast_actor_RMCE1.csv',index=False)

'''Dataset CREW. Crea una lista con los nombres de las columnas que se van a eliminar y luego las elimina con drop. Exporta el
dataframe como un archivo csv que se guarda en el workspace'''
elim_col_dir = ['belongs_to_collection','genres','id','original_language','overview','popularity','production_companies',
                     'production_countries','runtime','spoken_languages','status','tagline','vote_average',
                     'vote_count','release_year','cast']

movies_crew_dir = completo.drop(elim_col_dir,axis=1)
movies_crew_dir.to_csv('movies_crew_director_RMCE1.csv',index=False)

In [4]:
##--------- Prepara CSV  para realizar EDA'''
elim_col_sr = ['budget','revenue','return','original_language','popularity',
                     'release_date','runtime','spoken_languages','status','tagline','vote_average',
                     'vote_count','cast']
df_sr = completo.drop(elim_col_sr,axis=1)

# Crea nuevas columnas con los valores en forma de listas, para que se puedan procesar fácilmente despues:
#   Función para extraer los valores de una clave específica
def extract_values(dictionary_list, key):
    return [d[key] for d in dictionary_list]

#  Crea una nueva columna con los nombres de los géneros contenidos en una lista para cada película (línea) del dataframe
df_sr['genres_list'] = df_sr['genres'].apply(lambda x: extract_values(x, 'name'))

#  Crea una nueva columna con los nombres de los directores contenidos en una lista para cada película (línea) del dataframe
df_sr['directors'] = df_sr['crew'].apply(lambda x: extract_values(x, 'name'))

#   Crea una nueva columna con los nombres de las compañías productoras contenidos en una lista para cada película (línea) del dataframe
df_sr['production'] = df_sr['production_companies'].apply(lambda x: extract_values(x, 'name'))

#   Crea una nueva columna con los nombres de las compañías productoras contenidos en una lista para cada película (línea) del dataframe
df_sr['country'] = df_sr['production_countries'].apply(lambda x: extract_values(x, 'name'))

#   Crea una nueva columna llamada "collections", pero el contenido de la columna original puede tener 
#   distintas estructuras y/o contener o no la key "name", entonces hay que agregar condicionantes (if isinstance..etc)
df_sr['collection'] = df_sr[columna_dict].apply(lambda x: x[key1_dict] if isinstance(x, dict) and key1_dict in x else None)

# Sustituye NaN por un espacio vacío en los campos overview y collection
df_sr['overview'] = df_sr['overview'].fillna('')
 df_sr['collection'] = df_sr['collection'].fillna('') 

'''Lista con los nombres de las columnas que se van a eliminar del dataset dr_sr'''
col_to_elimin = ['belongs_to_collection','genres', 'id','crew','production_companies','production_countries']

'''     Elimina del dataframe las columnas especificadas'''
df_sr_csv = df_sr.drop(col_to_elimin, axis='columns')

df_sr_csv.to_csv('sis_rec_RMCE.csv',index=False)


In [9]:
#--------- Prepara CSV reducido para correr el sistema de recomendación en Render
# Convierte los valores a strings
df_sr_render = df_sr_csv.copy()
df_sr_render['genres_list'] = df_sr_render['genres_list'].apply(lambda x: ' '.join(x))
df_sr_render['directors'] = df_sr_render['directors'].apply(lambda x: ' '.join(x))
df_sr_render['production'] = df_sr_render['production'].apply(lambda x: ' '.join(x))
df_sr_render['country'] = df_sr_render['country'].apply(lambda x: ' '.join(x))

df_sr_render['collection'] = df_sr_render['collection'].fillna('') 

# Combinar las características de colección, género, director, casa productora y pais 
df_sr_render['features'] = df_sr_render['collection'] + ' ' + df_sr_render['genres_list'] + ' ' + df_sr_render['directors'] + ' ' + df_sr_render['production'] + ' ' + df_sr_render['country'] 

# Borra líneas al azar, se conserva una décima parte del dataframe original
random_indices = np.random.choice(df_sr_render.index, 44451, replace=False)
df_sr_render = df_sr_render.drop(random_indices)

'''Lista con los nombres de las columnas que se van a eliminar del dataset dr_sr'''
col_to_elimin = ['overview','release_year', 'genres_list','directors','production','country','collection']
'''     Elimina del dataframe las columnas especificadas'''
df_sr_render = df_sr_render.drop(col_to_elimin, axis='columns')

df_sr_render.to_csv('sis_rec_RMCE_reduced.csv',index=False)